In [ ]:
import polars as pl
import panel as pn
import panel.widgets as pnw
import ipywidgets as ipw
from ipyleaflet import Map, MarkerCluster, CircleMarker
pn.extension('plotly', 'ipywidgets')

In [ ]:
df = pl.read_parquet('data/nyc_taxi310k.parq')

In [ ]:
def make_map(pick=True):
    col_prefix = 'pick_' if pick else 'drop_'
    mean = [df[f'{col_prefix}lat'].mean(), df[f'{col_prefix}lon'].mean()]
    mapa = Map(center=mean, zoom=11, layout=ipw.Layout(width='400px', height='500px'))
    # preco body? budeme menit len atributy objektu mapa - center a body.markers
    body = MarkerCluster(markers=[])
    mapa.add(body)
    return mapa, body, mean

map_data = {'pick': make_map(pick=True), 'drop': make_map(pick=False)}

In [ ]:
day_choose = pnw.IntSlider(start=1, end=31, value=14, width=250, name='Deň')
hour_choose = pnw.IntSlider(start=0, end=23, value=11, width=300, name='Hodina')

In [ ]:
def view_map(day, hour, smer='pick'):
    col_prefix = 'pick_' if smer=='pick' else 'drop_'
    mapa, body, mean = map_data[smer]
    df_dh = df.filter((pl.col(f'{col_prefix}dt').dt.day() == day) & (pl.col(f'{col_prefix}dt').dt.hour() == hour))
    col_lat, col_lon = f'{col_prefix}lat', f'{col_prefix}lon'
    lat, lon = df_dh[col_lat], df_dh[col_lon]
    newcent = [lat.mean(), lon.mean()] if len(lat) else mean
    mapa.center = newcent
    marks = [CircleMarker(location=[lata, lona], radius=2) for lata, lona in zip(lat, lon)]
    body.markers = marks
    # mapa.zoom = 11
    return mapa

def pick_plot(day, hour):
    return view_map(day, hour, smer='pick')

def drop_plot(day, hour):
    return view_map(day, hour, smer='drop')

bind_pick = pn.bind(pick_plot, day=day_choose, hour=hour_choose)
bind_drop = pn.bind(drop_plot, day=day_choose, hour=hour_choose)

In [ ]:
pn.Column(pn.Row(day_choose, hour_choose), pn.Row(bind_pick, bind_drop)).servable()